### Identifica diferenças geométrias e temáticas entre várias ESRI Shapefiles ##

In [ ]:
from gasp.anls.ovlay import check_shape_diff

# Inputs

SHAPES_TO_COMPARE = {
    '/home/osmtolulc/webvia/coimbra/v11_cmb.shp'       : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v12_cmb.shp'       : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v13_cmb_2x2.shp'   : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v13_cmb_5x5.shp'   : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v13_cmb_10x10.shp' : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v14_cmb_2x2.shp'   : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v14_cmb_5x5.shp'   : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v14_cmb_10x10.shp' : "cls_int"
}

OUT_FOLDER = '/home/osmtolulc/webvia/cmb_anls'
REPORT     = '/home/osmtolulc/webvia/cmb_compare.xlsx'

conPARAM = {
    "HOST" : "localhost", "PORT" : "5432",
    "USER" : "postgres", "PASSWORD" : "admin", "TEMPLATE" : "template_postgis"
}

DB = "cmb_compare"

srs_code = 3857

RASTER_TEMPLATE = '/home/osmtolulc/webvia/boundaries/coimbra_20x20.shp'

check_shape_diff(SHAPES_TO_COMPARE, OUT_FOLDER, REPORT, conPARAM, DB, srs_code,
                GIS_SOFTWARE="GRASS", GRASS_REGION_TEMPLATE=RASTER_TEMPLATE)

### Union Analysis ###

In [ ]:
from gasp.anls.ovlay import union

LYR_A  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v13_coimbra.shp'
LYR_B  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v11_coimbra_clp.shp'
outSHP = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\teste_v13_v11.shp'
API    = 'saga'

union(LYR_A, LYR_B, outSHP, api_gis=API)

### Union Analysis in GRASS GIS ###

In [ ]:
from gasp.gt.wenv.grs import run_grass
from gasp.pyt.oss import fprop

GRS_WORKSPACE = r'D:\indev\OSM2LULC\WEB_Via\results_fev19'
LYR_A  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v13_coimbra.shp'
LYR_B  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v11_coimbra_clp.shp'
outSHP = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\grass_v13_v11.shp'

grsbase = run_grass(GRS_WORKSPACE, location='tst_union', srs=3763)
import grass.script.setup as gsetup
gsetup.init(grsbase, GRS_WORKSPACE, 'tst_union', 'PERMANENT')

# Import data
from gasp.gt.toshp.cff import shp_to_grs

lyr_a = shp_to_grs(LYR_A, fprop(LYR_A, 'fn'), asCMD=True)
lyr_b = shp_to_grs(LYR_B, fprop(LYR_B, 'fn'), asCMD=True)

import datetime
time_a = datetime.datetime.now().replace(microsecond=0)

from gasp.anls.ovlay import union
shpUnion = union(lyr_a, lyr_b, fprop(outSHP, 'fn'), api_gis="grass")

time_b = datetime.datetime.now().replace(microsecond=0)

print time_b - time_a

# Export data
from gasp.to.shp.grs import grs_to_shp
result = grs_to_shp(shpUnion, outSHP, "area")

## Optimized Union Analysis

In [ ]:
from gasp.anls.ovlay import optimized_union_anls

import datetime

time_a = datetime.datetime.now().replace(microsecond=0)
optimized_union_anls(
    "/home/osmtolulc/webvia/coimbra/v11_cmb.shp",
    "/home/osmtolulc/webvia/coimbra/v13_cmb_2x2.shp",
    "/home/osmtolulc/webvia/union_v11_v13_regular.shp",
    "/home/osmtolulc/webvia/boundaries/coimbra_20x20.shp", 3857, 
    multiProcess=None
)
time_b = datetime.datetime.now().replace(microsecond=0)

print time_b - time_a

In [ ]:
"""
Intersection GRASS GIS
"""

from gasp.gt.wenv.grs import run_grass
from gasp.pyt.oss import fprop

shp_a = '/home/jasp/mrgis/landsense_pp/tmp_lisboa/fishnets_shp/fishnet_35.shp'
shp_b = '/home/jasp/mrgis/landsense_pp/tstclip/cls1.shp'
output = '/home/jasp/mrgis/landsense_pp/tstclip/teste2.shp'
work = '/home/jasp/mrgis/landsense_pp/tstclip'
epsg = 32629


grsbase = run_grass(work, location='loc_int', srs=epsg)
import grass.script.setup as gsetup
gsetup.init(grsbase, work, 'loc_int', 'PERMANENT')

from gasp.gt.toshp.cff import shp_to_grs, grs_to_shp
from gasp.gt.gop.ovlay import intersection

shpa = shp_to_grs(shp_a, fprop(shp_a, 'fn'))
shpb = shp_to_grs(shp_b, fprop(shp_b, 'fn'))

# Intersection
intshp = intersection(shpa, shpb, fprop(output, 'fn'), api='grass')

# Export
result = grs_to_shp(intshp, output, 'area')

In [ ]:
"""
Clip GRASS GIS
"""

from gasp.gt.wenv.grs import run_grass
from gasp.pyt.oss import fprop

shp_shp = '/home/jasp/mrgis/landsense_pp/res_lisboa/cls_6.shp'
clip_shp = '/home/jasp/mrgis/landsense_pp/tmp_lisboa/boundaries/fishnet_22.shp'
output = '/home/jasp/mrgis/landsense_pp/testeclip.shp'
work = '/home/jasp/mrgis/landsense_pp'
epsg = 32629

grsbase = run_grass(work, location='loc_clip', srs=epsg)
import grass.script.setup as gsetup
gsetup.init(grsbase, work, 'loc_clip', 'PERMANENT')

from gasp.gt.toshp.cff import shp_to_grs, grs_to_shp
from gasp.gt.gop.ovlay import clip
from gasp.gt.prop.feat import feat_count

shp   = shp_to_grs(shp_shp, fprop(shp_shp, 'fn'))
clipf = shp_to_grs(clip_shp, fprop(clip_shp, 'fn'))

# Clip
clipshp = clip(shp, clipf, fprop(output, 'fn'), api_gis='grass')
print(feat_count(clipshp, gisApi='pygrass'))

# Export
result = grs_to_shp(clipshp, output, 'area')